# Configuração

Antes de começar este tutorial, conclua as seguintes etapas:

* Obtenha acesso ao Gemma fazendo login no [Hugging Face](https://huggingface.co/google/embeddinggemma-300M) e selecionando **Confirmar licença** para um modelo Gemma.

* Gere um [Token de Acesso](https://huggingface.co/docs/hub/en/security-tokens#how-to-manage-user-access-token) do Hugging Face e use-o para fazer login no Colab.

Este notebook será executado na CPU ou na GPU.

### Instalar depedencias

In [ ]:
!pip install -U sentence-transformers git+https://github.com/huggingface/transformers@v4.56.0-Embedding-Gemma-preview

# Embedding Gemma

In [ ]:
import os
import sys

os.environ["HF_TOKEN"] = ''

In [ ]:
import torch
#Use as bibliotecas `sentence-transformers` para criar uma instância de uma classe de modelo com o EmbeddingGemma.
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"

model_id = "google/embeddinggemma-300M"
model = SentenceTransformer(model_id).to(device=device)

print(f"Device: {model.device}")
print(model)
print("Total number of parameters in the model:", sum([p.numel() for _, p in model.named_parameters()]))

## Gerando embedding

Um embedding é uma representação numérica de um texto, como uma palavra ou frase, que captura seu significado semântico. Essencialmente, é uma lista de números (um vetor) que permite aos computadores compreender as relações e o contexto das palavras.

Vamos ver como EmbeddingGemma processaria três palavras diferentes `["Apple", "Maça", "Smartphone"]`.

EmbeddingGemma foi treinada em grandes quantidades de texto e aprendeu as relações entre palavras e conceitos.

In [ ]:
words = ["Apple", "Maça", "Smartphone"]
embeddings = model.encode(words)
print(embeddings)
for idx, embedding in enumerate(embeddings):
  print(f"Embedding {idx+1} (shape): {embedding.shape}")

O modelo gera um vetor numérico para cada frase. Os vetores reais são muito longos (768), mas para simplificar, são apresentados com algumas dimensões.

A chave não são os números individuais em si, mas **a distância entre os vetores**. Se representássemos esses vetores em um espaço multidimensional, os vetores para `maça` e `apple` estariam muito próximos um do outro. E o vetor para `smartphone` pode estar longe de `maça`, mas perto de `apple`.

## Liste tarefas disponíveis para fazer com esse modelo

In [ ]:
print("Tarefas disponíveis:")
for name, prefix in model.prompts.items():
  print(f" {name}: \"{prefix}\"")
print("-"*80)


## Determinando Similaridade

Nesta seção, usamos embeddings para determinar o quão sementicamente semelhantes são as diferentes sentenças. Aqui mostramos exemplos com pontuações de similaridade altas, médias e baixas.

Alta similaridade:
- Frase A: “O chef preparou uma refeição deliciosa para os convidados”.
- Frase B: “Um saboroso jantar foi preparado pelo chef para os visitantes”.
- Raciocínio: Ambas as frases descrevem o mesmo evento usando palavras e estruturas gramaticais diferentes (voz ativa vs. voz passiva). Eles transmitem o mesmo significado central.

Semelhança Média:
- Frase A: “Ela é especialista em aprendizado de máquina”.
- Frase B: “Ele tem um profundo interesse em inteligência artificial”.
- Raciocínio: As frases estão relacionadas porque o aprendizado de máquina é um subcampo da inteligência artificial. No entanto, eles falam sobre pessoas diferentes com diferentes níveis de envolvimento (especialista versus interesse).

Baixa similaridade:
- Frase A: "O tempo em Tóquio está ensolarado hoje."
- Frase B: “Preciso comprar mantimentos para a semana”.
- Raciocínio: As duas frases tratam de tópicos completamente não relacionados e não compartilham nenhuma sobreposição semântica.

In [ ]:
sentence_high = [
    "O chef preparou uma refeição deliciosa para os convidados.",
    "Um jantar saboroso foi preparado pelo chef para os visitantes."
]
sentence_medium = [
    "Ela é especialista em aprendizado de máquina.",
    "Ele tem um grande interesse em inteligência artificial."
]
sentence_low = [
    "O tempo em Tóquio está ensolarado hoje.",
    "Preciso comprar mantimentos para a semana."
]

for sentence in [sentence_high, sentence_medium, sentence_low]:
  print("🙋‍♂️")
  print(sentence)
  embeddings = model.encode(sentence)
  similarities = model.similarity(embeddings[0], embeddings[1])
  print("`-> 🤖 score: ", similarities.numpy()[0][0])

## Classificação
Nesta seção, usamos embeddings para classificar a qual área pertence cada sentença de uma situação problema.

In [ ]:
labels = ["Problema de Cobrança", "Suporte Técnico", "Consulta de Vendas"]

sentence = [
"Com licença, o aplicativo trava na tela de login. Não funciona nem quando tento redefinir minha senha.",
"Gostaria de consultar os preços e recursos do plano empresarial para uma equipe de 50 pessoas.",
]

# Calculate embeddings by calling model.encode()
label_embeddings = model.encode(labels, prompt_name="Classification")
embeddings = model.encode(sentence, prompt_name="Classification")

# Calculate the embedding similarities
similarities = model.similarity(embeddings, label_embeddings)
print(similarities)

idx = similarities.argmax(1)
print(idx)

for example in sentence:
  print("🙋‍♂️", example, "-> 🤖", labels[idx[sentence.index(example)]])

# Referencia
https://ai.google.dev/gemma/docs/embeddinggemma/inference-embeddinggemma-with-sentence-transformers?hl=pt-br